In [59]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
from os import getcwd, path, walk
import pandas as pd

In [67]:
from pokemon_card_generator.data import card_data
from pokemon_card_generator.data.set_data import get_sets
from pokemon_card_generator.predict import create_card
from pokemon_card_generator.data.card_data import create_card_data_pickle

In [ ]:
# regen pickle
create_card_data_pickle

In [62]:
cards_df = card_data.get_card_data()

In [63]:
pokémon_name = "Pikachu"

In [64]:
mask = cards_df.name == pokémon_name

In [65]:
pokémon_df = cards_df[mask]
pokémon_df

,id,name,subtypes,hp,types,evolvesFrom,weaknesses,convertedRetreatCost,rarity,flavorText,...,attack3_cost,attack3_converted_cost,attack3_damage,attack3_text,ability1_name,ability1_text,ability2_name,ability2_text,V_pokemon,Vmax_pokemon
57,base1-58,Pikachu,Basic,40.0,Lightning,NaN,Lightning,1.0,Common,"When several of these Pokémon gather, their el...",...,,,,,,,,,0,0
161,base2-60,Pikachu,Basic,50.0,Lightning,Pichu,Fire,1.0,Common,"When several of these Pokémon gather, their el...",...,,,,,,,,,0,0
166,basep-1,Pikachu,Basic,60.0,Lightning,NaN,Fighting,1.0,Promo,"When several of these Pokémon gather, their el...",...,,,,,,,,,0,0
169,basep-4,Pikachu,Basic,50.0,Lightning,NaN,Grass,1.0,Promo,"When several of these Pokémon gather, they att...",...,,,,,,,,,0,0
189,basep-24,Pikachu,Basic,50.0,Lightning,NaN,Lightning,1.0,Promo,Your Birthdate: ______________________________,...,,,,,,,,,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12183,swsh1-65,Pikachu,Basic,60.0,Lightning,NaN,No_weaknesses,1.0,Common,Pikachu that can generate powerful electricity...,...,,,,,,,,,0,0
12867,swsh4-43,Pikachu,Basic,190.0,Lightning,NaN,No_weaknesses,1.0,Rare Holo V,NaN,...,,,,,,,,,1,0
12868,swsh4-44,Pikachu,VMAX,310.0,Lightning,Pikachu V,No_weaknesses,2.0,Rare Holo VMAX,NaN,...,,,,,,,,,0,1
12994,swsh4-170,Pikachu,Basic,190.0,Lightning,NaN,No_weaknesses,1.0,Rare Holo V,NaN,...,,,,,,,,,1,0


In [9]:
create_card(pokémon_name = pokémon_name, rarity = 2)

{'data': {'id': '',
  'name': 'Pikachu',
  'supertype': 'Pokémon',
  'subtypes': [],
  'hp': '310',
  'types': ['Lightning'],
  'evolvesFrom': '',
  'abilities': [],
  'attacks': [{'name': 'Thundershock',
    'cost': ['Lightning'],
    'damage': '10',
    'text': 'Flip a coin. If heads, the Defending Pokémon is now Paralyzed.'},
   {'name': 'Tail Whap',
    'cost': ['Colorless', 'Colorless'],
    'damage': '20',
    'text': ''}],
  'weaknesses': [{'type': 'Lightning', 'value': 'x2'}],
  'resistances': [],
  'retreatCost': ['Colorless'],
  'convertedRetreatCost': 0,
  'set': {'id': '',
   'name': '',
   'series': '',
   'printedTotal': 0,
   'total': 0,
   'legalities': {'unlimited': 'Legal', 'expanded': 'Legal'},
   'ptcgoCode': '',
   'releaseDate': '',
   'updatedAt': '',
   'images': {'symbol': '', 'logo': ''}},
  'number': '0',
  'artist': '',
  'rarity': 2,
  'flavorText': "Pikachu's gender differences in the anime Pikachuuuuuuuu When do Volt Tackle and Double-Edge ever take a tur

In [56]:
def create_card_data_pickle():
    """Concatenates all the pickles in 'raw_data/pickles' and returns a dataframe.
    Usage: from pokemon_card_generator.data import card_data; card_df = card_data.get_card_data()"""
    # list all the files in data

    cwd = os.getcwd()
    two_dirs_up = path.dirname(path.dirname(cwd))
    pickles_path = path.join(two_dirs_up, "raw_data/pickles")
    print(pickles_path)

    filenames = next(walk(pickles_path), (None, None, []))[2]  # [] if no file

    # create a final df with first pickles
    cards_df = pd.read_pickle(path.join(pickles_path, filenames[0]))
    # open each other pickle and stack it into the final df
    for file in filenames[1:]:
        filepath = path.join(pickles_path, file)
        temp_df = pd.read_pickle(filepath)

        cards_df = pd.concat([cards_df, temp_df.reset_index()], axis=0)

    cards_df.reset_index(inplace=True)
    cards_df.drop(columns=["level_0"], inplace=True)

    ## add release dates in a new column
    sets_df = get_sets()
    release_date_dict = {k: v for k, v in zip(sets_df.id, sets_df.releaseDate)}
    card_sets = [row["id"].split("-")[0] for _, row in cards_df.iterrows()]
    card_release_dates = pd.Series([release_date_dict[set_] for set_ in card_sets])
    cards_df["releaseDate"] = card_release_dates

    ## Remove the list from types and fill in the first type
    types = []
    for type_ in cards_df.types:
        try:
            types.append(type_[0])
        except:
            types.append(type_)
    cards_df.types = pd.Series(types)

    ## Clean up weaknesses
    weaknesses = []
    for weakness in cards_df.weaknesses:
        try:
            weaknesses.append(weakness[0]['type'])
        except:
            weaknesses.append(weakness)

    cards_df.weaknesses = pd.Series(weaknesses)
    return cards_df

In [57]:
current_df = create_card_data_pickle()

/home/phil/code/pokemon_card_generator/raw_data/pickles


In [58]:
pika_df = current_df[current_df.name == "Pikachu"]
pika_df.weaknesses

57       Fighting
161      Fighting
166      Fighting
169      Fighting
191      Fighting
           ...   
11891    Fighting
12000    Fighting
12035    Fighting
12054    Fighting
12183    Fighting
Name: weaknesses, Length: 76, dtype: object

In [52]:
weaknesses = []
for weakness in card_df.weaknesses:
    try:
        weaknesses.append(weakness[0]['type'])
    except:
        weaknesses.append(weakness)
weaknesses

['Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 nan,
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting',
 'Fighting']